In [1]:
import puzzlerepresentation as P

In [2]:
import numpy as np

In [3]:
history = {}

In [4]:
def addChromosomeToHistory(C):
    history[C.tostring()] = -np.infty

In [5]:
h = (6,0)
null_C = np.zeros([10,3],dtype=np.int8)
if h:
    null_C[-1] = [h[0], h[1], 0]
addChromosomeToHistory(null_C)

In [6]:
def buildRandomChromosome(hole=None):
    C = np.random.randint(0, 7, size=(10,3), dtype=np.int8)
    C[:,-1] = np.random.randint(0,4, size=10, dtype=np.int8)
    if hole:
        C[-1] = [hole[0], hole[1], 0]
    if C.tostring() in history:
        return buildRandomChromosome(hole=hole)
    addChromosomeToHistory(C)
    return C

In [7]:
h = (6,0)

In [8]:
pieces = P.pieces + [P.Piece('black',[[0,0]])]

In [9]:
def score(C):
    A = np.zeros([7,7],dtype=np.int8)
    #pieces = P.pieces + [P.Piece('black',[[0,0]])]
    for i in range(10):
        for unit in pieces[i].rot_list(C[i,2]):
            x = C[i][0] + unit[0]
            y = C[i][1] + unit[1]
            if (min(x,y) < 0) or (max(x,y) > 6):
                return np.infty
            A[x][y] += 1
    #print(np.abs(A-1).sum())
    #print(A)
    s = np.abs(np.square(A-1)).sum()
    history[C.tostring()] = s
    return s

In [10]:
def rebuildChromosome(C, hole=None):
    C = C.reshape(10,3)
    
    C[:,:-1] = C[:,:-1] % 7
    C[:, -1] = C[:, -1] % 4
    
    return C

In [11]:
def stepChromosomes(C, hole=None):
    old_C = C.flatten()
    C_list = []
    iter_size = old_C.size
    if hole:
        iter_size -=3
    for x in range(iter_size):
        new_C_add = old_C.copy()
        new_C_sub = old_C.copy()
        
        new_C_add[x] += 1
        new_C_sub[x] -= 1
        
        new_C_add = rebuildChromosome(new_C_add).copy()
        new_C_sub = rebuildChromosome(new_C_sub).copy()
        
        if new_C_add.tostring() not in history:
            addChromosomeToHistory(new_C_add)
            C_list.append(new_C_add.copy())
        
        if new_C_sub.tostring() not in history:
            addChromosomeToHistory(new_C_sub)
            C_list.append(new_C_sub.copy())
        
    return C_list

In [12]:
C_list = [buildRandomChromosome(hole=h) for _ in range(10000)]

In [13]:
best_C = null_C.copy()
best_score = score(best_C)

In [14]:
for C in C_list:
    s = score(C)
    if s < best_score:
        best_C = C.copy()
        best_score = s
print(best_score)

28


In [15]:
def startSearch(hole=None):
    C_list = [buildRandomChromosome(hole=hole) for _ in range(10000)]
    best_C = null_C.copy()
    best_score = score(best_C)
    
    for C in C_list:
        s = score(C)
        if s < best_score:
            best_C = C.copy()
            best_score = s
    print(f'Search initialized with best score of: {best_score}')
    return [best_C]

In [16]:
best_C_list = [best_C]

In [17]:
for i in range(20):
    C_list = []
    for C in best_C_list:
        C_list = C_list + stepChromosomes(C, hole=h)
    two_steps = []
    for C in C_list:
        two_steps = two_steps + stepChromosomes(C, hole=h)
    C_list = C_list + two_steps
    best_C_list = []
    if len(C_list) == 0:
        best_C_list = startSearch(hole=h)
        best_score=np.infty
        print('No remaining paths to search.  Restarting from new random positions.')
    for C in C_list:
        s = score(C)
        if s < best_score:
            best_C_list.append(C.copy())
            best_score = s
    print(best_score)
    print(f'Finished iteration {i}.  {len(history)} board positions have been searched.')

20
Finished iteration 0.  11504 board positions have been searched.
14
Finished iteration 1.  14244 board positions have been searched.
12
Finished iteration 2.  15638 board positions have been searched.
Search initialized with best score of: 26
No remaining paths to search.  Restarting from new random positions.
inf
Finished iteration 3.  25638 board positions have been searched.
18
Finished iteration 4.  27141 board positions have been searched.
14
Finished iteration 5.  29881 board positions have been searched.
10
Finished iteration 6.  31275 board positions have been searched.
8
Finished iteration 7.  32670 board positions have been searched.
8
Finished iteration 8.  34065 board positions have been searched.
Search initialized with best score of: 22
No remaining paths to search.  Restarting from new random positions.
inf
Finished iteration 9.  44065 board positions have been searched.
16
Finished iteration 10.  45568 board positions have been searched.
14
Finished iteration 11.  46

In [18]:
len(history)

79735